# PC 3 : Réduction de dimension - 16 juin 2025 - Solution

Dans ce notebook, nous allons effectuer une analyse en composantes principales d'un jeu de données décrivant les scores obtenus par les meilleurs athlètes ayant participé en 2004 à une épreuve de décathlon, aux Jeux Olympiques d'Athènes ou au Décastar de Talence.

Nous utiliserons pour cela la toolbox [`scikit-learn`](http://scikit-learn.org/stable/index.html). `scikit-learn` est un ensemble de librairies de machine learning très utilisée et qui implémente la plupart des algorithmes d'apprentissage automatique non-profond. Il s'agit d'un projet Open Source. 

Dans une deuxième partie, vous vous essayerez à implémenter l'ACP vous-mêmes.

## Librairies python 

Ce notebook a été créé avec les versions suivantes des librairies :
* numpy 2.2.4
* matplotlib 3.10.1
* pandas 2.2.3
* sklearn 1.6.1
* seaborn 0.13.2

Tout devrait bien se passer si vous êtes bien dans l'environnement conda `sdd2025` chargé grâce au fichier `environment.yml` à la racine du répertoire. 

Des différences de version peuvent expliquer des comportements inattendus (avertissements, messages d'erreurs, fonctionalités inexistantes) mais il n'est pas nécessaire a priori d'avoir exactement les versions listées ci-dessus. Si vous souhaitez vérifier les versions des différentes librairies que vous utilisez, vous pouvez utiliser le code suivant :
```python
import <nom du module>
print(<nom du module>.__version__)
```

### Import de numpy et matplotlib

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
plt.rc('font', **{'size': 12}) # règle la taille de police globalement pour les plots (en pt)

## 1. Données

Les données sont contenues dans le fichier `decathlon.txt`.

Le fichier contient 42 lignes et 13 colonnes.

La première ligne est un en-tête qui décrit les contenus des colonnes.

Les lignes suivantes décrivent les 41 athlètes.

Les 10 premières colonnes contiennent les scores obtenus aux différentes épreuves.
La 11ème colonne contient le classement.
La 12ème colonne contient le nombre de points obtenus.
La 13ème colonne contient une variable qualitative qui précise l'épreuve (JO ou Décastar) concernée.

Nous allons examiner ces données avec la librairie `pandas`.

In [3]:
import pandas as pd
my_data = pd.read_csv('data/decathlon.txt', sep="\t")  # lire les données dans un dataframe

FileNotFoundError: [Errno 2] No such file or directory: 'data/decathlon.txt'

In [ ]:
my_data.head()

## 2. Quelques rappels sur `pandas`

Assurez-vous que vous comprenez l'usage des fonctionalités ci-dessous

In [ ]:
# Lister les noms des colonnes
my_columns = list(my_data.columns) 
print(my_columns)

In [ ]:
# Sélectionner une colonne
my_data['100m'].head()

In [ ]:
# Sélectionner plusieurs colonnes
columns = ['100m', '400m']
my_data[columns].head()

In [ ]:
# Sélectionner des lignes à partir de la valeur d'une colonne
my_data[my_data['Competition']=='OlympicG'].head()

In [ ]:
# Dimensions des données
print(my_data.shape)  

In [ ]:
# Valeurs du tableau sous forme d'array numpy
my_data_values = my_data.values
print(type(my_data_values))
print(my_data_values.shape)

In [ ]:
# Type des colonnes
print(my_data.dtypes)  

In [ ]:
# Nombre d'occurrences de chaque valeur unique dans une colonne
my_data['Competition'].value_counts()

## 3. Exploration des données

### 3.1 Distributions des variables
Visualisons la distribution des 10 variables continues représentant les performances aux 10 épreuves (c'est à dire les 10 premières variables) grâce à des histogrammes :

In [ ]:
fig = plt.figure(figsize=(12, 5))

# Histogrammes pour les variables continues
for (feat_idx, feat_name) in enumerate(my_data.columns[:10]):
    # créer une sous-figure (subplot) à la position (feat_idx+1) d'une grille 2x5
    ax = fig.add_subplot(2, 5, (feat_idx+1))

    # afficher l'histogramme de la variable feat_name
    h = ax.hist(my_data[feat_name], bins=10, edgecolor='none')

    # utiliser le nom de la variable comme titre pour chaque histogramme
    ax.set_title(feat_name)
    
# espacement entre les subplots
fig.tight_layout(pad=1.0)

Nous pouvons aussi visualiser la distribution de ces variables en fonction de l'épreuve :

In [ ]:
fig = plt.figure(figsize=(12, 5))

# Histogrammes pour les variables continues
for (feat_idx, feat_name) in enumerate(my_data.columns[:10]):
    # créer une sous-figure (subplot) à la position (feat_idx+1) d'une grille 2x5
    ax = fig.add_subplot(2, 5, (feat_idx+1))

    # afficher l'histogramme de la variable feat_name pour Competition=OlympicG
    h = ax.hist(my_data[my_data['Competition']=='OlympicG'][feat_name], bins=10,  
                color='tab:blue', edgecolor='none', alpha=1, label='Olympic Games')
    
    # afficher l'histogramme de la variable feat_name pour Competition=Decastar
    h = ax.hist(my_data[my_data['Competition']=='Decastar'][feat_name], bins=10,  
                color='tab:orange', edgecolor='none', alpha=0.8, label='Decastar')

    # utiliser le nom de la variable comme titre pour chaque histogramme
    ax.set_title(feat_name)

# Légende
plt.legend()

# espacement entre les subplots
fig.tight_layout(pad=1.0)

__Question :__ Les deux compétitions vous semblent-elles avoir des différences ?

__Réponse :__ Les distributions sont sensiblement les mêmes ; il y a plus d'entrées pour les JOs que pour le Decastar, on voit donc plus de valeurs « extrêmes » pour les JOs (l'histogramme bleu est un peu plus « étalé » que l'histogramme orange).

### 3.2. Couples de variables
Représentons les différents athlètes selon leur performance au 400m et au lancer du poids (`Shot.put`)

In [ ]:
fig = plt.figure(figsize=(5, 5)) # forcer une figure carrée

plt.scatter(my_data['400m'], my_data['Shot.put'], s=50)

plt.xlabel('Performance au 400m')
plt.ylabel('Performance au lancer du poids')

Les fourchettes de valeur prises par ces deux variables sont différentes, leurs unités aussi. Pour mieux les comparer il est souhaitable de les standardiser.

In [ ]:
fig = plt.figure(figsize=(5, 5)) # forcer une figure carrée

data_400m_std = (my_data['400m'] - np.mean(my_data['400m']))/(np.std(my_data['400m']))
data_shotput_std = (my_data['Shot.put'] - np.mean(my_data['Shot.put']))/(np.std(my_data['Shot.put']))

plt.scatter(data_400m_std, data_shotput_std, s=50)

plt.xlabel('400m (performance standardisée)')
plt.ylabel('Lancer du poids (performance standardisée)')

__Question :__ Ce nuage de points vous semble-t-il suggérer que les deux performances sont corrélées ?

__Réponse :__ Non (cf. les exemples à la fin du Chapitre 2 du poly)

#### Alternative avec seaborn
La librairie `seaborn` permet des visualisations plus élaborées que `matplotlib`. Vous pouvez par exemple explorer les capacités de [jointplot](https://seaborn.pydata.org/generated/seaborn.jointplot.html) :

In [ ]:
import seaborn as sns
# sns.set_style('whitegrid')

sns.jointplot(x='400m', y='Shot.put', data=my_data, 
              height=5, space=0, color='tab:blue')

#### Corrélation de deux variables

Nous pouvons calculer leur coefficient de corrélation de Pearson grâce au module `scipy.stats`. La fonction `pearsonr` renvoie même une p-valeur : https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html

In [ ]:
import scipy.stats as st

In [ ]:
r, pval = st.pearsonr(my_data['400m'], my_data['Shot.put'])
print(f"Corrélation entre le lancer du poids et le 400m : {r:.2f} (p={pval:.2e})")

__Question :__ Que signifie la valeur entre parenthèses précédée de `p=` ?

__Réponse :__ Il s'agit de la p-valeur d'un test statistique dont l'hypothèse nulle est l'absence de corrélation entre les deux variables.

__Question :__ Cela vous semble-t-il en accord avec le nuage de points ?

__Réponse :__ Oui. L'absence de corrélation est confirmée par la p-valeur.

__Question :__ Choisissez deux variables dont il vous semble vraisemblable qu'elles soient corrélées. Affichez le nuage de points correspondant. Calculez le coefficient de corrélation de Pearson. Souhaitez-vous rejeter votre hypothèse ?

__Réponse (exemple) :__

In [ ]:
r, pval = st.pearsonr(my_data['100m'], my_data['110m.hurdle'])
print(f"Corrélation entre le 100m et le 110m haies : {r:.2f} (p={pval:.2e})")

In [ ]:
fig = plt.figure(figsize=(4, 4)) # forcer une figure carrée

data_100m_std = (my_data['100m'] - np.mean(my_data['100m']))/(np.std(my_data['100m']))
data_hurdles_std = (my_data['110m.hurdle'] - np.mean(my_data['110m.hurdle']))/(np.std(my_data['110m.hurdle']))


plt.scatter(data_100m_std, data_hurdles_std, s=50)

plt.xlabel('100m (performance standardisée)')
plt.ylabel('110m haies (performance standardisée)')

Sans surprise, les performances au 110m haies et au 100 m sont corrélées.

#### Matrice de corrélation

Plutôt que de regarder les variables deux par deux, nous pouvons avoir une vue d'ensemble de leurs corrélations en affichant leur matrice de corrélation : 
* la méthode [DataFrame.corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html) de `pandas` nous permet de calculer cette matrice ;
* la méthode [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) de `seaborn` nous permet de la visualiser par une carte thermique (_heatmap_).

In [ ]:
# Calcul de la matrice de corrélation deux à deux
corr_matrix = my_data.drop(columns=['Points', 'Rank', 'Competition']).corr()

# Initialisation figure
plt.figure(figsize=(5, 4))

# Affichage heatmap
sns.heatmap(corr_matrix, 
            vmin=-1, # borne inf des valeurs à afficher
            vmax=1, # borne sup des valeurs à afficher
            center= 0, # valeur médiane des valeurs à afficher,
            cmap='PuOr', # colormap divergente de violet (PUrple) vers orange (ORange)
           )
# Titre
plt.title("Corrélation entre les performances aux 10 épreuves")

__Question :__ Quelles épreuves ont des performances très corrélées ? Quelles épreuves ont des performances indépendantes ?

__Réponse :__ 
* On retrouve la forte corrélation entre 100m et 110m haie (ainsi qu'avec le 400m), ou entre lancer de poids et lancer de disque.
* Les performances aux épreuves de sprint (100m, 110m haie et 400m) sont anti-corrélées aux performances de saut en longueur. (Remarquez qu'une bonne performance à la course est une valeur _faible_, tandis qu'une bonne performance au saut est une valeur _élevée_.)
* Les performances aux 110m haie sont indépendantes des performances au javelot, au saut à la perche ou au 1500m.

## 4. ACP des scores aux 10 épreuves

Nous allons maintenant effectuer une analyse en composantes principales des scores aux 10 épreuves.

### 4.1 Création de l'array numpy correspondant

In [ ]:
X = np.array(my_data.drop(columns=['Points', 'Rank', 'Competition']))
print(X.shape)

__Question :__ 
* Quel est le nombre d'individus dans la matrice de données `X` ?
* Quel est le nombre de variables dans la matrice de données `X` ?

__Réponse :__ 41 individus, 10 variables.

### 4.2 Standardisation des données

Plus haut, nous avons standardisé les variables nous-mêmes en leur retirant leur moyenne et en les divisant par leur écart-type. Cette tâche est entièrement automatisée par `scikit-learn` :
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
from sklearn import preprocessing

In [ ]:
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

### 4.3 Calcul des composantes principales

Les algorithmes de factorisation de matrice de `scikit-learn` sont inclus dans le module `decomposition`. Pour  l'ACP, référez-vous à : 
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
from sklearn import decomposition

Remarque : Nous avons ici peu de variables et pouvons nous permettre de calculer toutes les PC. 

La plupart des algorithmes implémentés dans `scikit-learn` suivent le fonctionnement suivant : 
* on instancie un objet, correspondant à un type d'algorithme/modèle, avec ses hyperparamètres (ici le nombre de composantes)
* on utilise la méthode `fit` pour passer les données à cet algorithme
* les paramètres appris sont maintenant accessibles comme arguments de cet objet.

In [ ]:
# Instanciation d'un objet PCA pour 10 composantes principales
pca = decomposition.PCA(n_components=10)
print(type(pca))

In [ ]:
# On passe maintenant les données standardisées à cet objet
# C'est ici que se font les calculs
pca.fit(X_scaled)

### 4.4 Proportion de variance expliquée par les PCs

Nous allons maintenant afficher la proportion de variance expliquée par les différentes composantes. Il est accessible dans le paramètre `explained_variance_ration_` de notre objet `pca`.

In [ ]:
plt.plot(np.arange(1, 11), pca.explained_variance_ratio_, marker='o')

plt.xlabel("Nombre de composantes principales")
plt.ylabel("Proportion de variance expliquée")

__Question :__ Affichez la proportion *cumulative* de variance expliquée

__Réponse :__

In [ ]:
# Réponse

np.cumsum(pca.explained_variance_ratio_)

plt.plot(np.arange(1, 11), np.cumsum(pca.explained_variance_ratio_), marker='o')

plt.xlabel("Nombre de composantes principales")
plt.title("Proportion cumulative de variance expliquée")

__Questions :__ 
* Quelle est la proportion de variance expliquée par les deux premières composantes ? 
* Combien de composantes faudrait-il utiliser pour expliquer 80% de la variance des données ?

__Réponse :__
* visuellement, 50% de la variance est expliquée par les 2 premières PC. On peut vérifier avec 
```python 
np.sum(pca.explained_variance_ratio_[:2])
```
ou
```python
np.cumsum(pca.explained_variance_ratio_)[1]
```
* 5 composantes (soit la moitié du nombre total de variables) permettent d'expliquer un peu plus que 80% de la variance des données. On peut vérifier avec
```python 
np.sum(pca.explained_variance_ratio_[:5])
```
ou
```python
np.cumsum(pca.explained_variance_ratio_)[4]
```

### 4.5 Projection des données sur les deux premières composantes principales

Nous allons maintenant utiliser uniquement les deux premières composantes principales.

Commençons par calculer la nouvelle représentation des données, c'est-à-dire leur projection sur ces deux PC. 

In [ ]:
pca = decomposition.PCA(n_components=2)
pca.fit(X_scaled)
X_projected = pca.transform(X_scaled)
print(X_projected.shape)

__Question :__ Affichez un nuage de points représentant les données selon ces deux PC.

__Réponse :__

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(X_projected[:, 0], X_projected[:, 1])

plt.xlabel("PC 1")
plt.ylabel("PC 2")

__Question :__ Colorez chaque point du nuage de points ci-dessus en fonction du classement de l'athlète qu'il représente. Qu'en conclure sur l'interprétation de la PC1 ?

__Réponse :__

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(X_projected[:, 0], X_projected[:, 1], c=my_data['Rank'])

plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.colorbar(label='classement')

Les meilleurs athlètes sont plutôt d'un côté du graphe et les moins bons de l'autre. La première composante principale traduit la performance globale des athlètes.

### 4.6 Interprétation des deux composantes principales
Chaque composante principale est une combinaison linéaire des variables décrivant les données. Les poids de cette combinaison linéaire sont accesibles dans `pca.components_`.

Nous pouvons visualiser non pas les individus, mais les 10 variables dans l'espace des 2 composantes principales.

In [ ]:
pcs = pca.components_
print(pcs[0])

In [ ]:
fig = plt.figure(figsize=(6, 6))

plt.scatter(pcs[0], pcs[1])
for (x_coordinate, y_coordinate, feature_name) in zip(pcs[0], pcs[1], my_data.columns[:10]):
    plt.text(x_coordinate, y_coordinate, feature_name)                          
    
plt.xlabel("Contribution à la PC1")
plt.ylabel("Contribution à la PC2")

__Question :__ Quelles variables ont des contributions très similaires aux deux composantes principales ? Qu'en déduire sur leur similarité ?

__Réponse :__ Les performances aux épreuves de 110m haie et de 100m, ou de lancer du poids et lancer du disque, sont très similaires. On imagine volontiers qu'elles requièrent les mêmes capacités physiques.

__Question :__ Comment interpréter le signe des contributions des variables à la première composantes principales ?

__Réponse :__  Les variables contribuant négativement à la PC1 correspondent aux épreuves où un bon score correspond à une petite valeur (on mesure un temps, il faut être rapide) tandis que celles contribuant positivement correspondent aux épreuves où un bon score correspond à une valeur élevée (on mesure un temps, il faut aller loin).

La première PC traduit ainsi la performance.

__Question facultative :__ Comment les valeurs de `pca.explained_variance_ratio_` sont-elles obtenues ? Refaites le calcul vous-mêmes.

In [ ]:
# Réponse possible
tot_var = np.var(X_scaled, axis=0).sum()
print((1 / tot_var) * np.var(X_projected, axis=0))

## 5 Implémentation de l'ACP
Cette partie s'inspire d'un notebook proposé par Joseph Boyd et Benoît Playe.

### 5.1 Décomposition spectrale de la matrice de covariance

Les deux premières composantes principales de X sont les deux vecteurs propres correspondant aux deux plus grandes valeurs propres de la matrice de covariance $\Sigma = \frac1n X^\top X.$

__Question :__ Utilisez `numpy.linalg` pour calculer les deux premières composantes principales de X. N'oubliez pas de travailler sur `X_scaled`. Comparez les à celles obtenues dans `pca.components_`.

In [ ]:
from numpy import linalg

# Calcul de la matrice de covariance
N = X_scaled.shape[0]
X_cov = X_scaled.T.dot(X_scaled) / N

# Décomposition spectrale
eigenvals, eigenvecs = linalg.eig(X_cov)
my_pcs = eigenvecs[:,:2]

In [ ]:
fig = plt.figure(figsize=(6, 6))

plt.scatter(my_pcs[:, 0], my_pcs[:, 1])
for (x_coordinate, y_coordinate, feature_name) in zip(my_pcs[:, 0], my_pcs[:, 1], my_data.columns[:10]):
    plt.text(x_coordinate, y_coordinate, feature_name)                          
    
plt.xlabel("Contribution à la PC1")
plt.ylabel("Contribution à la PC2")

Les résultats correspondent, même s'il est possible d'obtenir ici des vecteurs opposés à ceux données par `sklearn` (ce qui n'est pas anormal : les PCs sont uniques au sens près). On peut le vérifier numériquement :

__Question :__ Calculez la nouvelle représentation en deux dimensions des données. Comparez à celle obtenue avec `pca.transform`.

In [ ]:
np.sum(np.abs(my_pcs.T + pcs))

In [ ]:
my_new_X = X_scaled.dot(my_pcs)

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(my_new_X[:, 0], my_new_X[:, 1], c=my_data['Rank'])
                 #cmap=plt.get_cmap('viridis'))

#rank_first_indices = np.flatnonzero(my_data['Rank']==1)
#plt.scatter(X_projected[rank_first_indices, 0], X_projected[rank_first_indices, 1], label='Gagnants')

plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.colorbar(label='classement')

Les axes peuvent être inversés par rapport à la figure obtenue avec scikit-learn, si les PCs sont opposées.

### 5.2 Décomposition en valeurs singulières

__Question :__ Utiliser `linalg.svd` pour retrouver les deux premières composantes principales. Comparer.

In [ ]:
U, S, V = linalg.svd(X_scaled)

In [ ]:
svd_pcs = V[:2, :]

In [ ]:
pcs

In [ ]:
svd_pcs

Nous retrouvons les mêmes PCs qu'avec `sklearn`.